# T6. WALL RECONSTRUCTION

In this script, we reconstruct parametric wall geometries from the instance segmentation and reference heights.
Specifically, we need:
 - T2: instances of walls, ceilings and other objects
 - T5: reference levels

## LIBRARIES

In [1]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import json
from scipy.spatial.transform import Rotation   
import copy
import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

#import utils
from context import utils
import utils as utl
import utils.t1_utils as t1


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [4]:
name='beton_labels'

path=Path(os.getcwd()).parents[2]/'data'
pcd_input_path=os.path.join(path,f'{name}.laz')
class_file=path/'_classes.json'

name=name.split('_')[0]
json_output_path=os.path.join(path,f'{name}_walls.json') 
geometry_output_path= os.path.join(path,f'{name}_walls.obj') # these are the bounding surfaces of the reference levels (optional)

#bimfolder
# bimFolder=os.mkdir(path/name/'BIM')
graphPath=str(path/f'{name}Graph.ttl')

#thresholds
t_level=0.5
t_distance=0.7

Import Classes

In [5]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': -1, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 0, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 1, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 2, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 3, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 4, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 5, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


import Graph

In [6]:
graph=Graph().parse(graphPath)
nodes=tl.graph_to_nodes(graph)
wallNodes=[n for n in nodes if 'Walls' in n.subject]
print(f'{len(wallNodes)} wallNodes detected!')


29 wallNodes detected!


Import PCD

In [7]:
laz=laspy.read(pcd_input_path)

match point clouds with graph

In [8]:

for n in wallNodes: # this is 6 times slower than the code above !!
    idx=np.where((laz['classes']==n.class_id) & (laz['objects']==n.object_id))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(laz.xyz[idx])
    n.resource=pcd

In [9]:
# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(n.color) for n in wallNodes if n.resource is not None])
# o3d.visualization.draw_geometries([joined_pcd])

Import Reference Levels

In [10]:
levelNodes=[n for n in nodes if 'level' in n.subject]
referenceNodes=[]
for l in levelNodes:
    new_graph=ut.get_subject_graph(graph,levelNodes[0].subject)
    n=SessionNode(graph=new_graph)
    n.get_oriented_bounding_box()
    n.resource=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(n.orientedBoundingBox)
    referenceNodes.append(n) # something is wrong in the tl.graph_to_nodes function
levelNodes=referenceNodes
print(f'{len(levelNodes)} levelNodes detected!')

1 levelNodes detected!


## PROCESSING

Compute base constraint

In [11]:
for n in wallNodes:
    #compute minheight of the resource at 0.1% of the height (absolute minimum might be wrong)
    z_values = np.sort(np.asarray(n.resource.points)[:,2])
    minheight = np.percentile(z_values, 0.1)

    #compute base constraint. select the intersecting level that is closest to the bottom of the wall. Else, just take first levelNode.
    nearby_ref_levels= tl.select_nodes_with_intersecting_bounding_box(n,levelNodes)
    n.base_constraint= next((n for n in nearby_ref_levels if np.absolute(n.height-minheight)<t_level),levelNodes[0])  if nearby_ref_levels else levelNodes[0] #this is a link!
    
    #compute base offset
    n.base_offset=minheight-n.base_constraint.height
    print(f'name: {n.name}, base_constraint: {n.base_constraint.name}, base_offset: {n.base_offset}')


name: 2_Walls_43, base_constraint: level_00, base_offset: 0.07059945767538167
name: 2_Walls_44, base_constraint: level_00, base_offset: 0.0505994576753821
name: 2_Walls_45, base_constraint: level_00, base_offset: 2.690599457675381
name: 2_Walls_46, base_constraint: level_00, base_offset: 0.010599457675381174
name: 2_Walls_47, base_constraint: level_00, base_offset: 0.010599457675381174
name: 2_Walls_48, base_constraint: level_00, base_offset: 0.28059945767538075
name: 2_Walls_49, base_constraint: level_00, base_offset: 0.33059945767538146
name: 2_Walls_50, base_constraint: level_00, base_offset: 0.010599457675381174
name: 2_Walls_51, base_constraint: level_00, base_offset: -0.0094005423246184
name: 2_Walls_52, base_constraint: level_00, base_offset: 0.37490945767538086
name: 2_Walls_53, base_constraint: level_00, base_offset: 1.281599457675382
name: 2_Walls_54, base_constraint: level_00, base_offset: 0.16701945767538184
name: 2_Walls_55, base_constraint: level_00, base_offset: 0.180599

In [12]:
# o3d.visualization.draw_geometries([n.resource,o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(n.orientedBoundingBox)]+[levelNodes[0].resource])

Compute top constraint

In [13]:
for n in wallNodes:
    #compute maxheight of the resource at 0.1% of the height (absolute minimum might be wrong)
    z_values = np.sort(np.asarray(n.resource.points)[:,2])
    minheight = np.percentile(z_values, 0.1)
    maxheight = np.percentile(z_values, 99.9)

    #compute base constraint. select the intersecting level that is closest to the top of the wall. Else, just take last levelNode.
    nearby_ref_levels= tl.select_nodes_with_intersecting_bounding_box(n,levelNodes)
    n.top_constraint= next((n for n in nearby_ref_levels if np.absolute(n.height-maxheight)<t_level),levelNodes[-1]) if nearby_ref_levels else levelNodes[-1] #this is a link!
    
    #compute base offset
    n.top_offset=maxheight-n.top_constraint.height

    #compute wall height
    n.height=maxheight-minheight
    print(f'name: {n.name}, top_constraint: {n.top_constraint.name}, top_offset: {n.top_offset}')


name: 2_Walls_43, top_constraint: level_00, top_offset: 3.350599457675381
name: 2_Walls_44, top_constraint: level_00, top_offset: 3.230599457675382
name: 2_Walls_45, top_constraint: level_00, top_offset: 3.4505994576753807
name: 2_Walls_46, top_constraint: level_00, top_offset: 1.680599457675381
name: 2_Walls_47, top_constraint: level_00, top_offset: 1.9691194576754079
name: 2_Walls_48, top_constraint: level_00, top_offset: 3.2805994576753807
name: 2_Walls_49, top_constraint: level_00, top_offset: 4.280599457675381
name: 2_Walls_50, top_constraint: level_00, top_offset: 4.300599457675382
name: 2_Walls_51, top_constraint: level_00, top_offset: 2.4505994576753807
name: 2_Walls_52, top_constraint: level_00, top_offset: 4.300599457675382
name: 2_Walls_53, top_constraint: level_00, top_offset: 2.940599457675381
name: 2_Walls_54, top_constraint: level_00, top_offset: 2.7805994576753807
name: 2_Walls_55, top_constraint: level_00, top_offset: 2.5705994576753817
name: 2_Walls_56, top_constraint

In [14]:
# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in wallNodes if n.resource is not None])
# for n in wallNodes:
#     n.orientedBoundingBox.color=[1,0,0]
# o3d.visualization.draw_geometries([joined_pcd]+[n.orientedBoundingBox for n in wallNodes])

Compute Wall Orientation

In [15]:
for n in wallNodes:    
    #Compute the dominant plane on the point cloud
    n.plane_model, inliers = n.resource.segment_plane(distance_threshold=0.03,
                                            ransac_n=3,
                                            num_iterations=1000)
    
    #get center of the face and postion it on the correct height (base constraint + base offset)   
    n.faceCenter=n.resource.select_by_index(inliers).get_center()  
    n.faceCenter[2]=n.base_constraint.height + n.base_offset

    #compute the normal of the plane in 2D (third component should be zero)
    n.normal=n.plane_model[:3]
    n.normal[2]=0
    n.normal/=np.linalg.norm(n.normal)

    #temp pcd. this seems ok
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(np.asarray(n.resource.points)[inliers])
    n.hull, _ = pcd.compute_convex_hull()
    n.hull.paint_uniform_color([0,1,0])

    print(f'name: {n.name}, plane: {n.plane_model}, inliers: {len(inliers)}/{len(np.asarray(n.resource.points))}')      


name: 2_Walls_43, plane: [ 7.42136173e-01  6.70249133e-01  0.00000000e+00 -2.07484287e+05], inliers: 30595/31366
name: 2_Walls_44, plane: [ 7.34940143e-01  6.78131983e-01  0.00000000e+00 -2.08282275e+05], inliers: 7203/9829
name: 2_Walls_45, plane: [-6.60391485e-01  7.50921491e-01  0.00000000e+00 -7.76282577e+04], inliers: 881/1271
name: 2_Walls_46, plane: [ 7.50702824e-01  6.60640046e-01  0.00000000e+00 -2.06508769e+05], inliers: 2585/3088
name: 2_Walls_47, plane: [-6.69360310e-01  7.42937935e-01  0.00000000e+00 -7.51556984e+04], inliers: 11768/16149
name: 2_Walls_48, plane: [ 7.42767723e-01  6.69549184e-01  0.00000000e+00 -2.07428911e+05], inliers: 4555/7837
name: 2_Walls_49, plane: [ 7.46116295e-01  6.65815646e-01  0.00000000e+00 -2.07056665e+05], inliers: 4844/9533
name: 2_Walls_50, plane: [-6.64155108e-01  7.47594805e-01  0.00000000e+00 -7.66015225e+04], inliers: 23346/32940
name: 2_Walls_51, plane: [ 7.76644109e-01 -6.29939623e-01  0.00000000e+00  4.20228120e+04], inliers: 3061/5

Compute Wall thickness (this seems very subjective)

In [16]:
for n in wallNodes:
    #compute the normals of the wall
    pcd_tree = o3d.geometry.KDTreeFlann(n.resource)
    n.resource.estimate_normals() if not n.resource.has_normals() else None

    #for every 10th point in P, that has the same normal as the dominant plane, select nearest points in P that meet a distance threshold    
    points=np.asarray(n.resource.points)[::100]
    normals=np.asarray(n.resource.normals)[::100]
    idx=np.where(np.einsum('i,ji->j',n.normal,normals)>0.9)
    points=points[idx]
    normals=normals[idx]
    distances=[]

    for p,q in zip(points,normals):
        #compute distances
        [k, idx, _] = pcd_tree.search_radius_vector_3d(p, t_distance)        
        #retain only the distances for which the normal is within 0.7 radians of the normal of the point
        kNormals=np.asarray(n.resource.normals)[idx]
        indices=np.asarray(idx)[np.where(np.einsum('i,ji->j',q,kNormals)>0.9)]
        #compute the dotproduct between the point and the normals of the points in the radius
        vectors=p-np.asarray(n.resource.select_by_index(indices).points)
        #keep the max distance (orthogonal distance between the planes)
        distances.append(np.einsum('i,ji->j', q, vectors).max())

    #take the distance at the 99% percentile
    distance = np.percentile(np.sort(np.array(distances)), 99.9) 
    n.wallThickness=distance if distance > 0.1 else 0.1

    print(f'name: {n.name}, BB_extent: {n.orientedBoundingBox.extent}, wallThickness: {n.wallThickness}')


name: 2_Walls_43, BB_extent: [6.49056159 4.75590426 0.10778002], wallThickness: 0.179396053595281
name: 2_Walls_44, BB_extent: [3.27018629 1.36183811 0.26310474], wallThickness: 0.22641596503076483
name: 2_Walls_45, BB_extent: [0.94117845 0.89088775 0.40682449], wallThickness: 0.1
name: 2_Walls_46, BB_extent: [1.69050668 0.96558248 0.17433685], wallThickness: 0.1653791052825195
name: 2_Walls_47, BB_extent: [5.55492086 2.06237987 0.17774122], wallThickness: 0.1811781406860116
name: 2_Walls_48, BB_extent: [3.05043157 1.48686731 0.19833939], wallThickness: 0.36288113550752443
name: 2_Walls_49, BB_extent: [4.0895122  1.98675756 0.27309029], wallThickness: 0.1940692650377675
name: 2_Walls_50, BB_extent: [5.62433934 5.54840923 0.17348883], wallThickness: 0.21934579830688145
name: 2_Walls_51, BB_extent: [2.55619854 1.59419888 0.19475631], wallThickness: 0.34742087943302
name: 2_Walls_52, BB_extent: [11.42473702  4.53005322  0.23531543], wallThickness: 0.35861504266675337
name: 2_Walls_53, BB_

Compute Wall axis

In [17]:
for n in wallNodes:     
    
    # THIS DOESNT WORK FOR SINGLE FACED WALLS. compute the sign. This is given by the dotproduct of the normal of the face with the vector between the center of the face and the center of the oriented bounding box
    boxCenter=n.orientedBoundingBox.get_center()
    boxCenter[2]=n.base_constraint.height + n.base_offset
    n.sign=np.sign(np.dot(n.normal,n.faceCenter-boxCenter))

    #offset the center of the plane with half the wall thickness in the direction of the normal of the plane  
    wallCenter=n.faceCenter-n.sign*n.normal*n.wallThickness/2 
    wallCenter[2]=n.base_constraint.height + n.base_offset

    #project axis aligned bounding points on the plane
    box=n.resource.get_axis_aligned_bounding_box()    
    points=np.asarray(box.get_box_points())
    points[:,2]=n.base_constraint.height + n.base_offset

    #translate the points to the plane
    vectors=points-wallCenter
    translation=np.einsum('ij,j->i',vectors,n.normal)
    points=points - translation[:, np.newaxis] * n.normal

    # Calculate the pairwise distances between all boundary points
    distances = np.linalg.norm(points[:, np.newaxis] - points, axis=2)

    # Get the indices of the two points with the maximum distance
    max_indices = np.unravel_index(np.argmax(distances), distances.shape)

    # Retain only the two points with the maximum distance
    n.boundaryPoints = points[max_indices,:]

    #create the axis
    n.axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(n.boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1])

    # Calculate the length
    n.wallLength = np.linalg.norm(n.boundaryPoints[0] - n.boundaryPoints[1])

    print(f'name: {n.name}, wallLength: {n.wallLength}')


name: 2_Walls_43, wallLength: 5.875057558785933
name: 2_Walls_44, wallLength: 1.3299920431513088
name: 2_Walls_45, wallLength: 0.9036119762697203
name: 2_Walls_46, wallLength: 1.0488968625034996
name: 2_Walls_47, wallLength: 5.612638626173183
name: 2_Walls_48, wallLength: 1.5416219846920676
name: 2_Walls_49, wallLength: 1.802220740907902
name: 2_Walls_50, wallLength: 3.951298211899341
name: 2_Walls_51, wallLength: 1.516349734812245
name: 2_Walls_52, wallLength: 11.492814924496471
name: 2_Walls_53, wallLength: 1.2115652381582411
name: 2_Walls_54, wallLength: 1.5276557080696238
name: 2_Walls_55, wallLength: 1.574683657808287
name: 2_Walls_56, wallLength: 1.9848324732767064
name: 2_Walls_57, wallLength: 2.07995809807924
name: 2_Walls_58, wallLength: 2.117663651633936
name: 2_Walls_59, wallLength: 13.488557995734162
name: 2_Walls_60, wallLength: 1.791317126854431
name: 2_Walls_61, wallLength: 1.6917126326651843
name: 2_Walls_62, wallLength: 2.565300755466596
name: 2_Walls_63, wallLength: 2

In [18]:
# o3d.visualization.draw_geometries([joined_pcd]+
#                                   [n.orientedBoundingBox for n in wallNodes]+
#                                   [n.axis for n in wallNodes]+
#                                   gmu.join_geometries([o3d.geometry.PointCloud(o3d.utility.Vector3dVector(n.boundaryPoints)) for n in wallNodes]))

Compute Wall Geometry

In [19]:
for n in wallNodes:
    pointList=[]
    points=np.asarray(n.axis.points)
    pointList.extend(points+n.sign*n.normal*n.wallThickness/2)
    pointList.extend(points-n.sign*n.normal*n.wallThickness/2)
    pointList.extend(np.array(pointList)+np.array([0,0,n.height]))
    pcd=o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(pointList))

    box=pcd.get_oriented_bounding_box()
    n.wall=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(box)
    n.wall.paint_uniform_color(ut.literal_to_array(n.color))
    n.wallBox=o3d.geometry.LineSet.create_from_oriented_bounding_box(box)
    n.wallBox.paint_uniform_color([0,0,1])

    print(f'name: {n.name}, wall: {n.wall}')


name: 2_Walls_43, wall: TriangleMesh with 8 points and 12 triangles.
name: 2_Walls_44, wall: TriangleMesh with 8 points and 12 triangles.
name: 2_Walls_45, wall: TriangleMesh with 8 points and 12 triangles.
name: 2_Walls_46, wall: TriangleMesh with 8 points and 12 triangles.
name: 2_Walls_47, wall: TriangleMesh with 8 points and 12 triangles.
name: 2_Walls_48, wall: TriangleMesh with 8 points and 12 triangles.
name: 2_Walls_49, wall: TriangleMesh with 8 points and 12 triangles.
name: 2_Walls_50, wall: TriangleMesh with 8 points and 12 triangles.
name: 2_Walls_51, wall: TriangleMesh with 8 points and 12 triangles.
name: 2_Walls_52, wall: TriangleMesh with 8 points and 12 triangles.
name: 2_Walls_53, wall: TriangleMesh with 8 points and 12 triangles.
name: 2_Walls_54, wall: TriangleMesh with 8 points and 12 triangles.
name: 2_Walls_55, wall: TriangleMesh with 8 points and 12 triangles.
name: 2_Walls_56, wall: TriangleMesh with 8 points and 12 triangles.
name: 2_Walls_57, wall: TriangleMe

In [20]:
# o3d.visualization.draw_geometries([joined_pcd]+
#                                   [n.wallBox for n in wallNodes]+
#                                   [n.axis for n in wallNodes]+
#                                   [o3d.geometry.PointCloud(o3d.utility.Vector3dVector(n.boundaryPoints)) for n in wallNodes])

## EXPORT

json with metadata

In [21]:
#declare json
json_data = {
        "filename": ut.get_filename(json_output_path),
        "objects": []
    }
#fill json
for n in wallNodes:
    obj = {
            "name": n.name,
            "base_constraint":n.base_constraint.name,
            "base_offset":n.base_offset,
            "top_constraint":n.top_constraint.name,
            "top_offset":n.top_offset,
            "height": n.height,
            "wallThickness": n.wallThickness,
            "wallLength": n.wallLength,
            "normal": {
                "x": n.normal[0],
                "y": n.normal[1],
                "z": n.normal[2]
            },
            "startpoint": {
                "x": n.boundaryPoints[0][0],
                "y": n.boundaryPoints[0][1],
                "z": n.boundaryPoints[0][2]
            }
            ,
            "endpoint": {
                "x": n.boundaryPoints[1][0],
                "y": n.boundaryPoints[1][1],
                "z": n.boundaryPoints[1][2]
            }
            }
    json_data["objects"].append(obj)


obj with walls

In [27]:
def write_obj_with_submeshes(filename, meshes, mesh_names):
    """
    Write multiple Open3D TriangleMesh objects to a single OBJ file with submeshes.

    Parameters:
    - filename: str, the name of the output OBJ file.
    - meshes: list of open3d.geometry.TriangleMesh, the meshes to write.
    - mesh_names: list of str, the names of the submeshes.
    """
    if len(meshes) != len(mesh_names):
        raise ValueError("meshes and mesh_names must have the same length")

    vertex_offset = 1  # OBJ files are 1-indexed
    with open(filename, 'w') as file:
        for mesh, name in zip(meshes, mesh_names):
            file.write(f"g {name}\n")  # Start a new group for the submesh

            # Write vertices
            for vertex in mesh.vertices:
                file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")

            # Write faces, adjusting indices based on the current offset
            for triangle in mesh.triangles:
                adjusted_triangle = triangle + vertex_offset
                file.write(f"f {adjusted_triangle[0]} {adjusted_triangle[1]} {adjusted_triangle[2]}\n")

            # Update the vertex offset for the next mesh
            vertex_offset += len(mesh.vertices)

# Example usage:
# Assuming mesh1 and mesh2 are your Open3D TriangleMesh objects
mesh1 = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
mesh1.compute_vertex_normals()

mesh2 = o3d.geometry.TriangleMesh.create_box(width=1.0, height=1.0, depth=1.0)
mesh2.compute_vertex_normals()

write_obj_with_submeshes(geometry_output_path, [n.wall for n in wallNodes], [n.name for n in wallNodes])

graph with metadata

In [28]:
wallNodesBIM=[]
for n in wallNodes:
    b=BIMNode(subject=n.subject+'_BIM',
            derivedFrom=n.subject, #this should be a URI
            resource=n.wall,
            base_constraint=n.base_constraint,
            base_constraint_name=n.base_constraint.name,
            base_offset=n.base_offset,
            top_constraint=n.top_constraint,
            top_constraint_name=n.top_constraint.name,
            top_offset=n.top_offset,
            height=n.height,
            wallThickness=n.wallThickness,
            wallLength=n.wallLength,
            normal=n.normal,
            startpoint=n.boundaryPoints[0],
            endpoint=n.boundaryPoints[1],
            color=n.color)
    wallNodesBIM.append(b)
    print(b.to_graph().serialize())
new_graph=tl.nodes_to_graph(wallNodesBIM)
graph=graph+new_graph
print(graph.serialize(graphPath, format="turtle"))

@prefix e57: <http://libe57.org#> .
@prefix v4d: <https://w3id.org/v4d/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<file:///2_Walls_43_BIM> a v4d:BIMNode ;
    e57:cartesianBounds """[1.03772569e+05 1.03776640e+05 1.94657148e+05 1.94661628e+05
 9.22000000e+00 1.25000000e+01]""" ;
    e57:cartesianTransform """[[1.00000000e+00 0.00000000e+00 0.00000000e+00 1.03774604e+05]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 1.94659388e+05]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 1.08600000e+01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]]""" ;
    e57:pointCount 8 ;
    v4d:base_constraint "<geomapi.nodes.sessionnode.SessionNode object at 0x000002D1DCE3C340>" ;
    v4d:base_constraint_name "level_00" ;
    v4d:base_offset "0.07059945767538167"^^xsd:float ;
    v4d:color "[0.01568627 0.4        0.6745098 ]" ;
    v4d:derivedFrom "file:///2_Walls_43" ;
    v4d:endpoint "[1.03772635e+05 1.94661568e+05 9.22000000e+00]" ;
    v4d:faceCount 12 ;
    v4d:he